Задание 1
Скачайте с сайта https://grouplens.org/datasets/movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.


In [7]:
import chardet
with open(movies.dat, 'rb') as rd:
    result = chardet.detect(rd.read(100000))
print(result) # 'ISO-8859-1'

NameError: name 'movies' is not defined

In [1]:

import zipfile as z
import pandas as pd

zip_package = 'ml-1m.zip'
mov_in_zip = zip_package.strip('.zip') + '/movies.dat'
rat_in_zip = zip_package.strip('.zip') + '/ratings.dat'

# создание объекта с данным (это не str, а b"bytes\n", поэтому нужно перекодировать),
with z.ZipFile(zip_package, 'r') as data_in:
#     data_in.printdir()
#     print(data_in.namelist())

    # чтение файла с фильмами
    with data_in.open(mov_in_zip, 'r') as m:
        movies_list = [bytes.decode(line_, 'ISO-8859-1') for line_ in m.readlines()]
    
    # в pandas надо открывать файл, поэтому сохраняем в файл
    with open('movies_list.txt', 'w', encoding='utf-8') as out_file_1:
        out_file_1.writelines(''.join(movies_list))
    
    # то же самое для файла с рейтингами
    with data_in.open(rat_in_zip, 'r') as r:
        ratings_list = [bytes.decode(line_, 'ISO-8859-1') for line_ in r.readlines()]
  
    with open('ratings_list.txt', 'w', encoding='utf-8') as out_file_2:
        out_file_2.writelines(''.join(ratings_list))
    
# создаем датафреймы
with open('movies_list.txt', 'r', encoding='utf-8') as file_m:
    dfm = pd.read_csv(file_m, sep=r'::', names = ['MovieID', 'Title', 'Genres'], engine='python')
    
with open('ratings_list.txt', 'r', encoding='utf-8') as file_r:
    dfr = pd.read_csv(file_r, sep=r'::', names = ['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')

print(dfm.head)
print(dfr.head)

# далее надо сделать join по графе с MovieID через pd.concat
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html


<bound method NDFrame.head of       MovieID                               Title  \
0           1                    Toy Story (1995)   
1           2                      Jumanji (1995)   
2           3             Grumpier Old Men (1995)   
3           4            Waiting to Exhale (1995)   
4           5  Father of the Bride Part II (1995)   
...       ...                                 ...   
3878     3948             Meet the Parents (2000)   
3879     3949          Requiem for a Dream (2000)   
3880     3950                    Tigerland (2000)   
3881     3951             Two Family House (2000)   
3882     3952               Contender, The (2000)   

                            Genres  
0      Animation|Children's|Comedy  
1     Adventure|Children's|Fantasy  
2                   Comedy|Romance  
3                     Comedy|Drama  
4                           Comedy  
...                            ...  
3878                        Comedy  
3879                         Drama  


In [22]:
# dfm.head()
# dfm.info()
dfr.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   UserID     1000209 non-null  int64
 1   MovieID    1000209 non-null  int64
 2   Rating     1000209 non-null  int64
 3   Timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [27]:
len(dfr.MovieID.unique()) # уникальные фильмы
len(dfr.UserID.unique()) # уникальные пользователи
# dfr.groupby('UserID').count().sort_values('MovieID', ascending=False).head()
dfr.groupby('MovieID').count().sort_values('Rating', ascending=False).head()

,UserID,Rating,Timestamp
MovieID,,,
2858,3428,3428,3428
260,2991,2991,2991
1196,2990,2990,2990
1210,2883,2883,2883
480,2672,2672,2672


In [28]:
joined = dfr.merge(dfm, on='MovieID', how='left')
joined.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [178]:
joined.groupby('Title').count().sort_values('Rating', ascending=False).head()

,UserID,MovieID,Rating,Timestamp,Genres
Title,,,,,
American Beauty (1999),3428,3428,3428,3428,3428
Star Wars: Episode IV - A New Hope (1977),2991,2991,2991,2991,2991
Star Wars: Episode V - The Empire Strikes Back (1980),2990,2990,2990,2990,2990
Star Wars: Episode VI - Return of the Jedi (1983),2883,2883,2883,2883,2883
Jurassic Park (1993),2672,2672,2672,2672,2672


In [179]:
fltr_cls = joined[['Title', 'Rating']]
fltr_cls.head()

,Title,Rating
0,One Flew Over the Cuckoo's Nest (1975),5
1,James and the Giant Peach (1996),3
2,My Fair Lady (1964),3
3,Erin Brockovich (2000),4
4,"Bug's Life, A (1998)",5


In [189]:
filtered = fltr_cls.query('Rating == 5')
# filtered.tail(20)
filtered['Title'].value_counts().head(1)

American Beauty (1999)    1963
Name: Title, dtype: int64

In [ ]:
country_stats.query('quantity > {}'.format(average_level)).head()

In [6]:
import chardet
with open('movies.dat', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result # 'ISO-8859-1'

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

In [ ]:
?pd.read_csv

In [ ]:
# ratings.dat
# UserID::MovieID::Rating::Timestamp
# 1::122::5::838985046
# 1::185::5::838983525
# 1::231::5::838983392
# 1::292::5::838983421
# 1::316::5::838983392
# 1::329::5::838983392
# 1::355::5::838984474
# 1::356::5::838983653
# 1::362::5::838984885
# 1::364::5::838983707

In [ ]:
# movies.dat
# MovieID::Title::Genres
# 1::Toy Story (1995)::Adventure|Animation|Children|Comedy|Fantasy
# 2::Jumanji (1995)::Adventure|Children|Fantasy
# 3::Grumpier Old Men (1995)::Comedy|Romance
# 4::Waiting to Exhale (1995)::Comedy|Drama|Romance
# 5::Father of the Bride Part II (1995)::Comedy
# 6::Heat (1995)::Action|Crime|Thriller
# 7::Sabrina (1995)::Comedy|Romance
# 8::Tom and Huck (1995)::Adventure|Children
# 9::Sudden Death (1995)::Action
# 10::GoldenEye (1995)::Action|Adventure|Thriller

In [ ]:
''.join(str(movies_list[1:2])).split('::')


In [ ]:
data_ = pd.read_csv(str(movies_list), names = ['MovieID', 'Title', 'Genres'], sep =':', delimiter=':', header=0)
data_.head
# print(type(data_))
# dataframe = pd.DataFrame(data)


In [ ]:
mov_in_zip

Задание 2
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.


In [ ]:
# Lithuania
# Latvia
# Estonia

In [30]:
data = pd.read_csv('power.csv')
data.head()

,country,year,quantity,category
0,Austria,1996,5.0,1
1,Austria,1995,17.0,1
2,Belgium,2014,0.0,1
3,Belgium,2013,0.0,1
4,Belgium,2012,35.0,1


In [157]:
data.year.head()

10030

In [33]:
data['year'].head()

0    1996
1    1995
2    2014
3    2013
4    2012
Name: year, dtype: int64

In [47]:
country_stats = data.filter(items = ['country', 'quantity'])
country_stats.head()

,country,quantity
0,Austria,5.0
1,Austria,17.0
2,Belgium,0.0
3,Belgium,0.0
4,Belgium,35.0


In [50]:
data['quantity'].mean()
avg_lvl = data['quantity'].mean()
country_stats.query('quantity > {}'.format(avg_lvl)).head()

,country,quantity
3228,United States,367987.0
3229,United States,384439.0
3230,United States,370625.0
3231,United States,310909.0
3232,United States,335418.0


In [122]:
cntr_list = ['Lithuania', 'Latvia', 'Estonia']

In [144]:
# data[ data.quantity > avg_lvl ].head()
# data[ (data['country'] == 'Lithuania') & (2005 <= data['year']) & (data['year'] <= 2010) ].head(10)
fltr_cntrs = (data[ 
                    ((data['country'] == cntr_list[0]) |
                     (data['country'] == cntr_list[1]) |
                     (data['country'] == cntr_list[2])) &
                    (2005 <= data['year']) &
                    (data['year'] <= 2010) &
                    ((data['category'] == 4) |
                     (data['category'] == 12) |
                     (data['category'] == 21)) &
                    (data['quantity'] > 0)
                    ])
# fltr_cntrs = data[ (data['country'] == 'Latvia') | (data['country'] == 'Lithuania') | (data['country'] == 'Estonia') ].head()

In [159]:
# print(fltr_cntrs['country'].unique())
# len(fltr_cntrs)
print(fltr_cntrs.head(200))
print(sum(fltr_cntrs['quantity']))

          country  year  quantity  category
10617     Estonia  2008       1.0         4
10618     Estonia  2007       0.0         4
10619     Estonia  2006       0.0         4
10620     Estonia  2005       1.0         4
11156   Lithuania  2008       1.0         4
...           ...   ...       ...       ...
209276    Estonia  2006    2495.0        21
209277    Estonia  2005    2492.0        21
210472     Latvia  2010     951.0        21
210473     Latvia  2009     937.0        21
210474     Latvia  2008     575.0        21

[200 rows x 4 columns]
240481.0


In [71]:
sorted(data['country'].unique())

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctic Fisheries',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia (Plur. State of)',
 'Bonaire, St Eustatius, Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Rep.',
 'Chad',
 'Chile',
 'China',
 'China, Hong Kong SAR',
 'China, Macao SAR',
 'Colombia',
 'Commonwealth of Independent States (CIS)',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czechia',
 'Czechoslovakia (former)',
 "Côte d'Ivoire",
 'Dem. Rep. of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Domin

Задание 3
Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
https://www.finanz.ru/valyuty/v-realnom-vremeni


In [175]:
mifns9 = pd.read_html('https://www.nalog.gov.ru/rn77/about_fts/fts/structure_fts/mri_fns/mi_kn_9/', encoding='utf-8')

In [176]:
mifns9[0]

,Unnamed: 0,Время работы инспекции,Перерыв
0,ПН,9:00-18:00,13:00-13:45
1,ВТ,9:00-18:00,13:00-13:45
2,СР,9:00-18:00,13:00-13:45
3,ЧТ,9:00-18:00,13:00-13:45
4,ПТ,9:00-16:45,13:00-13:45
5,ВС,не работает,NaN
